In [ ]:
! pip install --upgrade --user --quiet google-cloud-aiplatform google-cloud-storage

In [ ]:
! pip install --upgrade --user --quiet tqdm pandas

In [4]:
! gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=vqSq7TVlcOc14BKfEy87alWA78jQ9T&access_type=offline&code_challenge=BWozrD7r8--4MkTFv5KZIAGcdyavzZ0UXoBkD6HitgE&code_challenge_method=S256


You are now logged in as [support@unvailed.co].
Your current project is [unvailed-466101].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [ ]:
! gcloud config list

In [1]:
import pandas as pd
from google.cloud import storage, aiplatform
from vertexai.preview.language_models import TextEmbeddingModel
import vertexai
import tqdm
import time

PROJECT_ID = "unvailed-466101"
LOCATION = "us-central1"
BUCKET_NAME = "unvailed_test_bucket_1"
CSV_FILE_PATH = "Unvailed Vendors - Supported.csv"

vertexai.init(project=PROJECT_ID, location=LOCATION)

df = pd.read_csv(CSV_FILE_PATH)

In [5]:
model = TextEmbeddingModel.from_pretrained("gemini-embedding-001")

def get_embeddings_wrapper(texts, batch_size=5):
    embeddings = []
    for i in tqdm.tqdm(range(0, len(texts), batch_size)):
        time.sleep(1) 
        batch_texts = texts[i:i+batch_size]
        batch_embeddings = model.get_embeddings(batch_texts)
        embeddings.extend([embedding.values for embedding in batch_embeddings])
    return embeddings

content = df['content'].tolist()
df['embedding'] = get_embeddings_wrapper(content)

GoogleAuthError: 
Unable to authenticate your request.
Depending on your runtime environment, you can complete authentication by:
- if in local JupyterLab instance: `!gcloud auth login` 
- if in Colab:
    -`from google.colab import auth`
    -`auth.authenticate_user()`
- if in service account or other: please follow guidance in https://cloud.google.com/docs/authentication